In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
import pandas as pd

from transformers import pipeline
import torch

In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
sweden = pd.read_csv('chinadaily_sweden_complete.csv')
sweden['article_body'] = sweden['text'].str.encode('utf-8').str.decode('unicode_escape')

In [4]:
pakistan = pd.read_csv('chinadaily_pakistan_complete.csv')
pakistan['article_body'] = pakistan['text'].str.encode('utf-8').str.decode('unicode_escape')

In [5]:
sweden.head()

,Unnamed: 0,title,keyword,file,url,text,img_links,fig_captions,date_cleaned,author,LLM_summary,LLM_hidden_meaning,article_body
0,0,'Decoupling' cannot make US stronger,"('Sweden',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202404/23/WS66...,While not much is expected from US Secretary o...,['//img2.chinadaily.com.cn/images/202404/23/66...,['Chinese and US flags flutter outside the bui...,2024-04-23 15:15,Harvey Dzodin,- **Main Topics:**\r\n1. US-China relations\r\...,Tone: The overall tone of the article is criti...,While not much is expected from US Secretary o...
1,1,'Freak' Haaland scores outrageous goal as City...,"('Sweden',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202410/25/WS67...,"MANCHESTER, England Erling Haaland scored an...",['//img2.chinadaily.com.cn/images/202410/25/67...,['Erling Haaland celebrates scoring Manchester...,2024-10-25 09:02,NaN,- **Main Topics:**\r\n * Erling Haaland\r\n...,**1. Tone and Style:**\nThe overall tone of th...,"MANCHESTER, England Erling Haaland scored an..."
2,2,'Green and just transition',"('Nordic',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202411/20/WS67...,Following the 2024 European Parliament electio...,['//img2.chinadaily.com.cn/images/202411/20/67...,"['WANG XIAOYING/CHINA DAILY', 'Yu Hongyuan', '...",2024-11-20 07:57,YU HONGYUAN and WANG XINYU,- **Main Topics:**\r\n\r\n1. European Union's ...,**Tone and Style:**\nThe overall tone is neutr...,Following the 2024 European Parliament electio...
3,3,2024 WLA Forum champions 'Excellence in Science',"('Nobel',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202410/24/WS67...,The 2024 World Laureates Association (WLA) For...,['//img2.chinadaily.com.cn/images/202410/24/67...,['Shanghai is set to host the 2024 World Laure...,2024-10-24 13:33,chinadaily.com.cn,- **Main Topics:**\r\n - World Laureates Asso...,**Tone and Style:**\nThe overall tone of the a...,The 2024 World Laureates Association (WLA) For...
4,4,2024 World Laureates Forum wraps up in Shanghai,"('Nobel',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202410/28/WS67...,The 2024 World Laureates Forum concluded in Sh...,['//img2.chinadaily.com.cn/images/202410/28/67...,"['Jon Kleinberg and Jeremy Nathans, winners of...",2024-10-28 19:53,China Daily in Shanghai,- **Main Topics:** \r\n - World Laureates F...,**1. Tone and Style:**\nThe overall tone of th...,The 2024 World Laureates Forum concluded in Sh...


In [6]:
custom_stopwords = [""]

def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())  # Lowercase and tokenize
    # Remove punctuation and stopwords
    stop_words = set(stopwords.words('english')).union(set(custom_stopwords))
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return tokens

# Tokenize all tweets
sweden.loc[:, 'TOKENS'] = sweden['article_body'].apply(preprocess_text)
pakistan.loc[:, 'TOKENS'] = pakistan['article_body'].apply(preprocess_text)

In [7]:
sweden.head()

,Unnamed: 0,title,keyword,file,url,text,img_links,fig_captions,date_cleaned,author,LLM_summary,LLM_hidden_meaning,article_body,TOKENS
0,0,'Decoupling' cannot make US stronger,"('Sweden',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202404/23/WS66...,While not much is expected from US Secretary o...,['//img2.chinadaily.com.cn/images/202404/23/66...,['Chinese and US flags flutter outside the bui...,2024-04-23 15:15,Harvey Dzodin,- **Main Topics:**\r\n1. US-China relations\r\...,Tone: The overall tone of the article is criti...,While not much is expected from US Secretary o...,"[much, expected, us, secretary, state, antony,..."
1,1,'Freak' Haaland scores outrageous goal as City...,"('Sweden',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202410/25/WS67...,"MANCHESTER, England Erling Haaland scored an...",['//img2.chinadaily.com.cn/images/202410/25/67...,['Erling Haaland celebrates scoring Manchester...,2024-10-25 09:02,NaN,- **Main Topics:**\r\n * Erling Haaland\r\n...,**1. Tone and Style:**\nThe overall tone of th...,"MANCHESTER, England Erling Haaland scored an...","[manchester, england, erling, haaland, scored,..."
2,2,'Green and just transition',"('Nordic',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202411/20/WS67...,Following the 2024 European Parliament electio...,['//img2.chinadaily.com.cn/images/202411/20/67...,"['WANG XIAOYING/CHINA DAILY', 'Yu Hongyuan', '...",2024-11-20 07:57,YU HONGYUAN and WANG XINYU,- **Main Topics:**\r\n\r\n1. European Union's ...,**Tone and Style:**\nThe overall tone is neutr...,Following the 2024 European Parliament electio...,"[following, 2024, european, parliament, electi..."
3,3,2024 WLA Forum champions 'Excellence in Science',"('Nobel',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202410/24/WS67...,The 2024 World Laureates Association (WLA) For...,['//img2.chinadaily.com.cn/images/202410/24/67...,['Shanghai is set to host the 2024 World Laure...,2024-10-24 13:33,chinadaily.com.cn,- **Main Topics:**\r\n - World Laureates Asso...,**Tone and Style:**\nThe overall tone of the a...,The 2024 World Laureates Association (WLA) For...,"[2024, world, laureates, association, wla, for..."
4,4,2024 World Laureates Forum wraps up in Shanghai,"('Nobel',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202410/28/WS67...,The 2024 World Laureates Forum concluded in Sh...,['//img2.chinadaily.com.cn/images/202410/28/67...,"['Jon Kleinberg and Jeremy Nathans, winners of...",2024-10-28 19:53,China Daily in Shanghai,- **Main Topics:** \r\n - World Laureates F...,**1. Tone and Style:**\nThe overall tone of th...,The 2024 World Laureates Forum concluded in Sh...,"[2024, world, laureates, forum, concluded, sha..."


In [12]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Assuming 'TOKENS' column has list of tokens
df = pakistan

# Combine tokens back to string format for CountVectorizer
df['clean_text'] = df['TOKENS'].apply(lambda tokens: ' '.join(tokens))

# Initialize CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=5, stop_words='english')
dtm = vectorizer.fit_transform(df['clean_text'])

# Initialize LDA
num_topics = 10
lda = LatentDirichletAllocation(
    n_components=num_topics,
    random_state=42,
    learning_method='batch',
    doc_topic_prior=0.0001,  # Learn alpha from data
    topic_word_prior=0.1     # Learn eta from data
)

# Fit LDA
lda.fit(dtm)

# Function to display topics
def display_topics(model, feature_names, no_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" | ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print()

# Display the topics
display_topics(lda, vectorizer.get_feature_names_out(), no_top_words=10)


Topic 1:
global | said | brics | world | xi | countries | china | international | south | development

Topic 2:
china | said | space | satellite | beijing | high | chinese | technology | africa | innovation

Topic 3:
olympic | games | world | sports | china | olympics | said | team | sport | gold

Topic 4:
said | china | people | chinese | countries | international | united | culture | foreign | pakistan

Topic 5:
china | countries | development | cooperation | global | international | world | bri | new | road

Topic 6:
china | chinese | said | xinjiang | pakistan | people | university | media | life | international

Topic 7:
economic | sanctions | countries | china | world | trade | united | states | diplomacy | international

Topic 8:
pakistan | china | sco | peace | economic | projects | chinese | cpec | meeting | islamabad

Topic 9:
trade | business | hainan | free | china | said | khan | company | percent | investment

Topic 10:
china | countries | people | government | rights | i

In [9]:
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-5-english")

def sentiment_analysis(text):
    # Truncate text to 512 tokens
    max_length = 512
    truncated_text = text[:max_length]

    result = sentiment_pipeline(truncated_text)[0]
    label = result['label']
    score = result['score']

    if label == 'POSITIVE':
        return 1,score
    elif label == 'NEGATIVE':
        return -1,score
    else:
        return 0

pakistan["Bert_model_code"] = pakistan["clean_text"].apply(sentiment_analysis)
sweden["Bert_model_code"] = sweden["clean_text"].apply(sentiment_analysis)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [22]:
count_minus_one_high_score = sweden['Bert_model_code'].apply(
    lambda x: x[0] == -1 and x[1] > 0.95
).sum()

print("Number of -1 values with score above 0.95:", count_minus_one_high_score/len(sweden['Bert_model_code']))

Number of -1 values with score above 0.95: 0.15606936416184972


In [21]:
count_minus_one_high_score = sweden['Bert_model_code'].apply(
    lambda x: x[0] == 1 and x[1] > 0.95
).sum()

print("Number of +1 values with score above 0.95:", count_minus_one_high_score/len(sweden['Bert_model_code']))

Number of +1 values with score above 0.95: 0.6416184971098265


### 3 classes of sentiment analysis

In [49]:
# Load the model fine-tuned on SST-5 (5 classes)
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
#nlptown/bert-base-multilingual-uncased-sentiment

# Function to classify sentiment and map to 5 classes
def sentiment_analysis(text):
    # Truncate text to 512 tokens
    max_length = 512
    truncated_text = text[:max_length]

    result = sentiment_pipeline(truncated_text)[0]
    label = result['label']
    score = result['score']

    # Map the 3 classes
    if label == 'LABEL_0':
        return 'negative', score
    elif label == 'LABEL_1':
        return 'neutral', score
    elif label == 'LABEL_2':
        return 'positive', score
    else:
        return None, score


Device set to use cpu


In [50]:
# Apply sentiment analysis to the 'clean_text' column for both datasets
pakistan[['sentiment_label', 'sentiment_score']] = pakistan['text'].apply(
    lambda x: pd.Series(sentiment_analysis(x))
)

sweden[['sentiment_label', 'sentiment_score']] = sweden['text'].apply(
    lambda x: pd.Series(sentiment_analysis(x))
)

In [51]:
pakistan

,Unnamed: 0,title,keyword,file,url,text,img_links,fig_captions,date_cleaned,author,LLM_summary,LLM_hidden_meaning,article_body,TOKENS,clean_text,Bert_model_code,sentiment_label,sentiment_score
0,0,'Chinese Bridge': A bridge for the world to China,"('Pakistani',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202410/14/WS67...,"During the global finals of the 23rd ""Chinese ...",['//img2.chinadaily.com.cn/images/202410/14/67...,"['', '', '', '', '', '', '', '', '', '', '', '...",2024-10-14 13:13,chinadaily.com.cn,"- **Main Topics:** \r\n - Global finals of ""C...",Here's the nuanced analysis:\n\n**Tone:** The ...,"During the global finals of the 23rd ""Chinese ...","[global, finals, 23rd, ``, chinese, bridge, ''...",global finals 23rd `` chinese bridge '' chines...,"(-1, 0.6303477883338928)",positive,0.588519
1,1,'Lion Forest Garden in My Eyes' International ...,"('Pakistani',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202411/04/WS67...,"From October 29 to 31, 2024, the ""Lion Forest ...",['//img2.chinadaily.com.cn/images/202411/04/67...,"['From October 29 to 31, 2024, the ""Lion Fores...",2024-11-04 15:32,chinadaily.com.cn,- **Main Topics:** \r\n - International Commu...,**Tone and Style:**\nThe tone of the article i...,"From October 29 to 31, 2024, the ""Lion Forest ...","[october, 29, 31, 2024, ``, lion, forest, gard...",october 29 31 2024 `` lion forest garden eyes ...,"(1, 0.9984649419784546)",positive,0.751316
2,2,'Lion Forest Garden in My Eyes' held to foster...,"('Pakistani',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202411/09/WS67...,"From October 29 to 31, 2024, the ""Lion Forest ...",[],[],2024-11-09 12:53,chinadaily.com.cn,- **Main Topics:** \r\n1. International Commun...,**Tone and Style:**\nThe overall tone of the a...,"From October 29 to 31, 2024, the ""Lion Forest ...","[october, 29, 31, 2024, ``, lion, forest, gard...",october 29 31 2024 `` lion forest garden eyes ...,"(1, 0.9987616539001465)",positive,0.725727
3,3,1st reusable satellite payloads returned,"('Pakistan',)",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202410/25/WS67...,Scientific payloads returned from the maiden f...,['//img2.chinadaily.com.cn/images/202410/25/67...,['China\'s first reusable satellite returns to...,2024-10-25 09:07,Zhao Lei,- **Main Topics:**\r\n* China's reusable satel...,**Tone and Style:**\n- Neutral tone\n- Formal ...,Scientific payloads returned from the maiden f...,"[scientific, payloads, returned, maiden, fligh...",scientific payloads returned maiden flight chi...,"(-1, 0.7172546982765198)",neutral,0.710617
4,4,8 stories from 8 years in China,"('Pakistan', 'Pakistani', 'Karachi', 'Punjab')",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/201404/03/WS5a...,Please click to read more My China Stories 1) ...,[],[],2014-04-03 16:52,Dr.Muhammad Shahbaz (Ali),The text appears to be an autobiography or per...,What an incredible collection of stories! Here...,Please click to read more My China Stories 1) ...,"[please, click, read, china, stories, 1, amazi...",please click read china stories 1 amazing thin...,"(-1, 0.9652535319328308)",positive,0.602134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,98,Yangling agri fair brings innovation to doorstep,"('Pakistan', 'Pakistani')",C:\Users\ondre\Desktop\semester3\Mining of Soc...,https://www.chinadaily.com.cn/a/202410/30/WS67...,With an expanded global reach and commitment t...,['//img2.chinadaily.com.cn/images/202410/30/67...,['A view of the 31st China Yangling Agricultur...,2024-10-30 09:03,LI JIAYING in Beijing and QIN FENG in Xi'an,- **Main Topics:**\r\n - China Yangling Agri...,Here is the nuanced analysis:\n\n**Tone and St...,With an expanded global reach and commitment t...,"[expanded, global, reach, commitment, pushing,...",expanded global reach commitment pus

In [56]:
# **Summary Statistics**: Count of each sentiment label
sent_reduced = sweden[sweden['sentiment_score'] >= 0.80]

sentiment_counts = sent_reduced['sentiment_label'].value_counts()

# **Average sentiment score** per class
avg_sentiment_scores = sweden.groupby('sentiment_label')['sentiment_score'].mean()

# Print summary statistics
print("Sentiment Counts:")
print(sentiment_counts)

print("\nAverage Sentiment Scores:")
print(avg_sentiment_scores)

Sentiment Counts:
sentiment_label
positive    49
neutral      9
negative     1
Name: count, dtype: int64

Average Sentiment Scores:
sentiment_label
negative    0.663174
neutral     0.655093
positive    0.766496
Name: sentiment_score, dtype: float64


In [57]:
# **Summary Statistics**: Count of each sentiment label
sent_reduced = pakistan[pakistan['sentiment_score'] >= 0.80]

# **Summary Statistics**: Count of each sentiment label
sentiment_counts = sent_reduced['sentiment_label'].value_counts()

# **Average sentiment score** per class
avg_sentiment_scores = pakistan.groupby('sentiment_label')['sentiment_score'].mean()

# Print summary statistics
print("Sentiment Counts:")
print(sentiment_counts)

print("\nAverage Sentiment Scores:")
print(avg_sentiment_scores)

Sentiment Counts:
sentiment_label
positive    24
neutral      6
negative     4
Name: count, dtype: int64

Average Sentiment Scores:
sentiment_label
negative    0.714296
neutral     0.665110
positive    0.768318
Name: sentiment_score, dtype: float64
